In [6]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import math
import time
import sys
import itertools
import random
import os
import signal
import gc
import psutil
from sklearn.manifold import TSNE

def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()


np.set_printoptions(threshold=sys.maxsize,linewidth=512)
#np.set_printoptions(threshold=256)

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("<style>.output_result { max-width:95% !important; }</style>"))
 
#여백 줄이기
display(HTML("<style>.prompt { min-width: 30ex !important; }</style>"))

In [7]:
# import EarlyStopping
# from pytorchtools import EarlyStopping
class EarlyStopping:
    def __init__(self, patience=10, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 10
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [8]:
target_length=100
unit_space_x=5
unit_space_y=5


In [9]:
def random_normal_int(size):
    sigma=size/4
    m=size/2
    min=1
    max=size-1
    a=0
    while a <=0 or a>=size-1:
        a=math.floor(sigma*np.random.randn()+m)
    return a

In [10]:
def cal_flat(active,y,x):
    x_dim = active.shape[1]
    y_dim = active.shape[0]
    coord = y*x_dim + x
    return coord

def cal_unflat(active,num):
    x_dim = active.shape[1]
    y_dim = active.shape[0]
    y = int(num / x_dim)
    x = num % x_dim
    return y,x

In [11]:
#####tools for calculating distance to interconnect

def find_start_end_mid(active_routing):
    a1=np.where(active_routing==1)[1]
    unique_elements, counts=np.unique(a1, return_counts=True)
    mid=unique_elements[counts>1][0]
    
    y1=np.where(active_routing==2)[0][1]
    y2=np.where(active_routing==2)[0][0]
    x1=np.where(active_routing==2)[1][1]
    x2=np.where(active_routing==2)[1][0]
    start= np.array([[y1,x1]])
    end = np.array([[y2,x2]])
    mid1 = np.array([[y1,mid]])
    mid2 = np.array([[y2,mid]])
    return start,end,mid1,mid2



In [12]:
def find_div_pos(divide,start,end,mid1,mid2):
    div_pos=np.copy(start)
    line1_length = (np.linalg.norm(start[0]-mid1[0]))*unit_space_x
    line2_length = (np.linalg.norm(mid1[0]-mid2[0]))*unit_space_y
    line3_length = (np.linalg.norm(mid2[0]-end[0]))*unit_space_x
    line_num=np.array([])
    for i in range(divide.shape[0]):
            
            if divide[i]<=line1_length :
                temp2 = np.copy(start)
                temp2[0][1] += divide[i]/unit_space_x
                div_pos = np.append(div_pos,temp2,axis=0)
                line_num=np.append(line_num,np.array([1]))
            if line1_length<divide[i]<=line1_length+line2_length:
                temp2 = np.copy(mid1)
                temp2[0][0] -= (divide[i]-line1_length)/unit_space_y
                div_pos = np.append(div_pos,temp2,axis=0)
                line_num=np.append(line_num,np.array([2]))

            if divide[i] > line1_length+line2_length:
                temp2 = np.copy(mid2)
                temp2[0][1] += (divide[i]-line1_length-line2_length)/unit_space_x
                div_pos = np.append(div_pos,temp2,axis=0)
                line_num=np.append(line_num,np.array([3]))

                
    div_pos = np.delete(div_pos,0,axis=0)
    return div_pos,line_num

In [13]:
def divide_line(active,min_rpt,start,end,mid1,mid2,add_num=0):
#    unit_space_x = int(target_x/active.shape[1])
#    unit_space_y = int(target_y/active.shape[0])
#    print("distance:",distance)
#    print("minimum # of repeaters:",min_rpt)
#    print("input:",x1,y1)
#    print("output:",x2,y2)
#    print("mid:",mid)
    min_rpt = min_rpt + add_num
    distance = np.sum(abs(start-end)*unit_space_x)    
    unit_distance = int(distance / (min_rpt+1) )
    divide = np.array([])
    divide_line = np.array([])
    for i in range(min_rpt):
        divide = np.append(divide, np.array(int(unit_distance/unit_space_x)*unit_space_x*(i+1)))
        
    div_pos,line_num = find_div_pos(divide,start,end,mid1,mid2)
    
    space=np.min(np.append(divide,distance)-np.append(0,divide))
    bound= int((target_length-space)/unit_space_x)        
    online_min=bound
    for i in range(div_pos.shape[0]-1):
        dist=int(math.dist(div_pos[i],div_pos[i+1]))
        if dist<online_min:
            online_min=dist
    return divide,div_pos,line_num,online_min

In [14]:
def extract_node_features(active,active_inout,active_routing):
    
    start,end,mid1,mid2=find_start_end_mid(active_routing)
#    divide,div_pos,line_num,online_min =divide_line(active,rpt_num,start,end,mid1,mid2,add_num=0)
    ## create feature matrix (node_num x features) => In this case, ( 32x64, 6)
    ## Features : Y-pos, X-pos, Active, Start/End, Interconnect, dist1, dist2, dist3, dist4
    node_num=active.shape[0]*active.shape[1]
    feature = np.zeros([node_num,9])
#    print(feature.shape)    
    ## insert 1st, 2nd features Y,X
    for i in range(active.shape[0]):
        for j in range(active.shape[1]):
            order=cal_flat(active,i,j)
            feature[order][1]= j
            feature[order][0]= i

    ## insert 3~5 feature Active / Start,End / Interconnect
    active_all = active_inout.flatten()
    active_all[active_all==0]= 1
    active_all[active_all==-1]=0
    active_all[active_all==2]=0
    train_mask = active_all
    test_mask = active_all
    
    active_all = active_inout.flatten()    
    active_all[active_all==-1]=1
    active_all[active_all==2]=1
    feature[:,2]= active_all
    
    active_all = active_routing.flatten()
    active_all[active_all==1]=0
    active_all[active_all==2]=1
    feature[:,3] = active_all
    
    active_all = active_routing.flatten()
    active_all[active_all==2]=0
    feature[:,4] = active_all
    
    start,end,mid1,mid2=find_start_end_mid(active_routing)

    ## insert 5~8th feature distance from metal lines
    dist1_y=abs(feature[:,0]-start[0][0])
    dist1_x=abs(feature[:,1]-start[0][1])
    dist2_y=abs(feature[:,0]-mid1[0][0])
    dist2_x=abs(feature[:,1]-mid1[0][1])
    dist3_y=abs(feature[:,0]-mid2[0][0])
    dist3_x=abs(feature[:,1]-mid2[0][1])
    dist4_y=abs(feature[:,0]-end[0][0])
    dist4_x=abs(feature[:,1]-end[0][1])

    
    
    
    feature[:,5]=dist1_y+dist1_x
    feature[:,6]=dist2_y+dist2_x
    feature[:,7]=dist3_y+dist3_x
    feature[:,8]=dist4_y+dist4_x
    
    return feature, train_mask, test_mask

In [ ]:
active=r_input[1,:,:,0]
active_inout = r_input[1,:,:,1]
active_routing = r_input[1,:,:,2]

rpt_num = np.argwhere(r_output_num==1)[:,1]+1

feature,train_mask,test_mask = extract_node_features(active,active_inout,active_routing)
start,end,mid1,mid2=find_start_end_mid(active_routing)
divide,div_pos,line_num,online_min =divide_line(active,rpt_num[1],start,end,mid1,mid2,add_num=0)

In [ ]:
rpt_num.shape

In [ ]:
print(start,end,mid1)

In [10]:
def extract_edges(active,active_inout,active_routing,rpt_num):
    start,end,mid1,mid2=find_start_end_mid(active_routing)
    divide,div_pos,line_num,online_min =divide_line(active,rpt_num,start,end,mid1,mid2,add_num=0)


    ###### Extract edges of connected lines
    y=start[0][0]
    x=start[0][1]

    src_node=np.array([])
    des_node=np.array([])
    for i in range(10000):
        x+=1
        if x<active.shape[1] and active_routing[y,x]==1:
            src_node=np.append(src_node,cal_flat(active,y,x-1))
            des_node=np.append(des_node,cal_flat(active,y,x))
        else:
            x-=1
            break

    for i in range(10000):
        y-=1
        if (y>=0 and active_routing[y,x]==1):
            src_node=np.append(src_node,cal_flat(active,y+1,x))
            des_node=np.append(des_node,cal_flat(active,y,x))
        else:
            y+=1
            break

    for i in range(10000):
        x+=1
        if x<active.shape[1] and (active_routing[y,x] in [1,2]):
            src_node=np.append(src_node,cal_flat(active,y,x-1))
            des_node=np.append(des_node,cal_flat(active,y,x))
        else:
            break

    edge_weights = np.ones(src_node.shape[0])
#    print(src_node.shape)
#    print(des_node.shape)
    


    
    ####### Extract edges of 0 active region to start,mid1,mid2,end
#    for i in range(4):
#        a=np.array(cal_flat(active,arr[0],arr[1]))
#        src_node=np.append(src_node,a)
#    for i in range(arr.shape[1]):
#        des_node=np.append(des_node,cal_flat(active,start[0][0],start[0][1]))
#    for i in range(arr.shape[1]):
#        des_node=np.append(des_node,cal_flat(active,mid1[0][0],mid1[0][1]))
#    for i in range(arr.shape[1]):
#        des_node=np.append(des_node,cal_flat(active,mid2[0][0],mid2[0][1]))
#    for i in range(arr.shape[1]):
#        des_node=np.append(des_node,cal_flat(active,end[0][0],end[0][1]))

#    start2=start.reshape(2,1)
#    mid1_2=mid1.reshape(2,1)
#    mid2_2=mid2.reshape(2,1)
#    end2=end.reshape(2,1)
#    edge_weights=np.append(edge_weights, np.sqrt(np.sum((arr-start2)**2,axis=0)))
#    edge_weights=np.append(edge_weights, np.sqrt(np.sum((arr-mid1_2)**2,axis=0)))
#    edge_weights=np.append(edge_weights, np.sqrt(np.sum((arr-mid2_2)**2,axis=0)))
#    edge_weights=np.append(edge_weights, np.sqrt(np.sum((arr-end2)**2,axis=0)))
#    edge_weights=edge_weights.reshape(-1,1)
#    print(edge_weights.shape)

    
    ########Extract edges < div_pos  -  Active 0 > ######
    arr=np.array(np.where(active_inout == 0))

    for i in range(div_pos.shape[0]):
        
        src_node=np.append(src_node,cal_flat(active,arr[0],arr[1]))
        des_node=np.append(des_node,cal_flat(active,div_pos[i][0],div_pos[i][1]).repeat(arr.shape[1]))
        dist_pos=np.sqrt(np.sum((arr-div_pos[i].reshape(2,1))**2,axis=0))
        dist_pos[dist_pos==0]=1
        dist_pos = 1/dist_pos
        edge_weights = np.append(edge_weights,dist_pos)

#    print(src_node.shape)
#    print(des_node.shape)    
#    print(edge_weights.shape)

    
    edges=np.append(src_node,des_node).reshape(2,-1)
    
    return edges,edge_weights

In [ ]:
for i in range(100):
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
##### output for gnn ###
r_output_modify = np.copy(r_input[:,:,:,0])
print(r_output_modify.shape)

for i in range(r_output_modify.shape[0]):
    r_output_modify[i][r_output_modify[i]==-1]=3
    coord=np.argwhere(r_output[i]==1)
    for j in range(coord.shape[0]):
        r_output_modify[i,coord[j][0],coord[j][1]]=4
print(r_output_modify.shape)

In [11]:
r_input=np.load("./gnn/dataset3/input500k.npy")[:10000]
r_input_pos = np.load("./gnn/dataset3/input500k_pos.npy")[:10000]
r_output=np.load("./gnn/dataset3/output500k.npy")[:10000]
r_output_num = np.load("./gnn/dataset3/output500k_num.npy")[:10000]
r_output_pos = np.load("./gnn/dataset3/output500k_pos_flatten.npy")[:10000]
print(r_input.shape)
print(r_input_pos.shape)
print(r_output.shape)
print(r_output_num.shape)
print(r_output_pos.shape)

(10000, 32, 64, 3)
(10000, 32, 64, 3)
(10000, 32, 64)
(10000, 10)
(10000, 2048)


In [ ]:
#np.save("./gnn/dataset3/output500k_pos.npy",r_output_modify.astype(np.int8))
print(r_output_num[0])

In [12]:
from torch_geometric.data import Data
import torch

gc.collect()

torch.cuda.empty_cache()

def gen_graph(r_input,r_output_pos,r_output_num):
    rpt_num = np.argwhere(r_output_num==1)[:,1]+1
    
    dataset=[]
    for i in range(r_input.shape[0]):

        active=r_input[i,:,:,0]
        active_inout=r_input[i,:,:,1]
        active_routing=r_input[i,:,:,2]
        node_feature,train_mask,test_mask=extract_node_features(active,active_inout,active_routing)
        edges,edge_weights=extract_edges(active,active_inout,active_routing,rpt_num[i])
        
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        node_feature = torch.from_numpy(node_feature).to(device)
        edges= torch.from_numpy(edges).to(device)
        edge_weights = torch.from_numpy(edge_weights).to(device)
        r_output_modify = torch.from_numpy(r_output_pos[i]).to(device)

        node_feature = node_feature.to(dtype=torch.float32)
        edges = edges.to(dtype=torch.long)
        edge_weights = edge_weights.to(dtype=torch.float32)
        r_output_modify = r_output_modify.to(dtype=torch.long)
        
#        r_output_modify = torch.tensor(np.sum(r_output[i]==1)).to(dtype=torch.long).to(device)
        data=Data(
            x=node_feature,
            edge_index=edges,
            y=r_output_modify,
            edge_weight=edge_weights,
            train_mask=train_mask,
            test_mask=test_mask,
        )
#        data.num_classes=len(torch.unique(data.y))
        data.num_classes=2       
#        data.edge_weight = edge_weights
#        data.train_mask = train_mask
#        data.test_mask = test_mask
        dataset.append(data)
    
    return dataset



In [13]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv,GATConv,GATv2Conv,SAGEConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(44)
        
        self.conv = torch.nn.ModuleList()
        self.conv.append(GCNConv(9, 64))
        for i in range(10):
            self.conv.append(GCNConv(64,64))
        
        self.out = Linear(64, 1)
        

    def forward(self, data):
        x, edge_index,edge_weights = data.x, data.edge_index, data.edge_weight
        
        for conv in self.conv:
            x = conv(x, edge_index, edge_weight=edge_weights)
            x = F.relu(x)
            x = F.dropout(x, p=0.5, training = self.training)
        
#        x = F.softmax(self.out(x),dim=1)
        x= torch.sigmoid(self.out(x))
#        x = torch.cat([gmp(x, batch_index)])
        
#        return F.softmax(x,dim=1)
        x= x.squeeze()
        return x

#edge_weight=edge_weights
model = GCN()

model = model.to(device)
model.eval()

GCN(
  (conv): ModuleList(
    (0): GCNConv(9, 64)
    (1): GCNConv(64, 64)
    (2): GCNConv(64, 64)
    (3): GCNConv(64, 64)
    (4): GCNConv(64, 64)
    (5): GCNConv(64, 64)
    (6): GCNConv(64, 64)
    (7): GCNConv(64, 64)
    (8): GCNConv(64, 64)
    (9): GCNConv(64, 64)
    (10): GCNConv(64, 64)
  )
  (out): Linear(in_features=64, out_features=1, bias=True)
)

In [14]:
import torch
import torch.nn.functional as F
from sklearn.utils import class_weight

def weighted_binary_crossentropy(y_true, y_pred):
    epsilon = 1e-7
    
#    y_true = torch.reshape(torch.tensor(y_true), [-1])
#    y_pred = torch.reshape(torch.tensor(y_pred), [-1])
    y_true_np = y_true.clone().detach().cpu().numpy().astype('int32')
    y_true_float = y_true.float().requires_grad_(True)
    y_pred_float = y_pred.float().requires_grad_(True)
    class_weights = torch.tensor(class_weight.compute_class_weight(class_weight='balanced', classes=[0, 1], y=y_true_np), dtype=torch.float32)
    y_pred_float = torch.clamp(y_pred_float, epsilon, 1 - epsilon)
    loss = -y_true_float * torch.log(y_pred_float) * class_weights[1] - (1 - y_true_float) * torch.log(1 - y_pred_float) * class_weights[0]
    
    # Alternatively, you can use PyTorch's built-in binary cross-entropy loss with sample weights
    # bce = F.binary_cross_entropy(y_pred_float, y_true_float, weight=class_weights)
    # loss = torch.mean(bce)
    
    return torch.mean(loss)

In [15]:
#from pytorchtools import EarlyStopping
#early_stopping = EarlyStopping(patience=10, verbose=True, delta=0.001)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)
#criterion = torch.nn.CrossEntropyLoss()
#criterion = torch.nn.BCELoss()

def train(dataset):
    
    model.train()
    optimizer.zero_grad()  # Clear gradients.
    out = model(dataset)  # Perform a single forward pass.
#    loss = criterion(out[dataset.train_mask].reshape(-1), dataset.y[dataset.train_mask].float())  # Compute the loss solely based on the training nodes.
    loss = weighted_binary_crossentropy(dataset.y, out)
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss


def test(dataset):
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        val_acc = 0.0
        out2 = model(dataset)
        val_loss = weighted_binary_crossentropy(dataset.y,out2)
    return val_loss



In [ ]:
del model
gc.collect()
torch.cuda.empty_cache()

model = GCN()

model = model.to(device)

In [16]:
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split

dataset=gen_graph(r_input,r_output.reshape(r_output.shape[0],-1),r_output_num)
len(dataset)

batchs = 64
epochs = 10

early_stopping = EarlyStopping(patience = 20, verbose = True)

# Define the sizes of the training and validation (or test) sets
train_size = int(0.9 * len(dataset)) # 90% of the dataset will be used for training
val_size = len(dataset) - train_size # the remaining 20% of the dataset will be used for validation

# Use the random_split function to split the dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Define the data loaders for the training and validation (or test) sets
train_loader = DataLoader(train_dataset, batch_size=batchs, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batchs, shuffle=False)



#loader=DataLoader(dataset, batch_size =batchs, shuffle=True)

In [17]:
train_loss=0
val_loss=0
for epoch in range(1, 500):
    for batch in train_loader:
  
        loss = train(batch)
        train_loss += loss
        
    for batch2 in val_loader:
        val_loss = test(batch2)
        val_losses += val_loss
    train_losses /= len(train_loader)
    val_losses /= len(val_loader)
    
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {train_losses:.4f}, Val_Loss: {val_losses:.4f}')



Epoch: 001, Loss: 0.7655, Val_Loss: 0.6936
Epoch: 002, Loss: 0.6905, Val_Loss: 0.6928
Epoch: 003, Loss: 0.6896, Val_Loss: 0.6927
Epoch: 004, Loss: 0.6204, Val_Loss: 0.6929
Epoch: 005, Loss: 0.5995, Val_Loss: 0.6933
Epoch: 006, Loss: 0.6225, Val_Loss: 0.6940
Epoch: 007, Loss: 0.5876, Val_Loss: 0.6951
Epoch: 008, Loss: 0.5903, Val_Loss: 0.6964
Epoch: 009, Loss: 0.5781, Val_Loss: 0.6981
Epoch: 010, Loss: 0.5631, Val_Loss: 0.7001
Epoch: 011, Loss: 0.5677, Val_Loss: 0.7027
Epoch: 012, Loss: 0.5919, Val_Loss: 0.7060
Epoch: 013, Loss: 0.5615, Val_Loss: 0.7102
Epoch: 014, Loss: 0.5545, Val_Loss: 0.7155
Epoch: 015, Loss: 0.5999, Val_Loss: 0.7225
Epoch: 016, Loss: 0.5378, Val_Loss: 0.7316
Epoch: 017, Loss: 0.4858, Val_Loss: 0.7436
Epoch: 018, Loss: 0.5152, Val_Loss: 0.7585
Epoch: 019, Loss: 0.4942, Val_Loss: 0.7774
Epoch: 020, Loss: 0.5113, Val_Loss: 0.8002
Epoch: 021, Loss: 0.5458, Val_Loss: 0.8266
Epoch: 022, Loss: 0.4933, Val_Loss: 0.8576
Epoch: 023, Loss: 0.5260, Val_Loss: 0.8952
Epoch: 024,

Epoch: 192, Loss: 0.2555, Val_Loss: 1.9051
Epoch: 193, Loss: 0.3033, Val_Loss: 1.8956
Epoch: 194, Loss: 0.2189, Val_Loss: 1.9241
Epoch: 195, Loss: 0.2843, Val_Loss: 1.9290
Epoch: 196, Loss: 0.2431, Val_Loss: 1.9271
Epoch: 197, Loss: 0.2205, Val_Loss: 1.9178
Epoch: 198, Loss: 0.2576, Val_Loss: 1.9203
Epoch: 199, Loss: 0.2266, Val_Loss: 1.9107
Epoch: 200, Loss: 0.2633, Val_Loss: 1.9273
Epoch: 201, Loss: 0.2577, Val_Loss: 1.9515
Epoch: 202, Loss: 0.2709, Val_Loss: 1.9579
Epoch: 203, Loss: 0.2392, Val_Loss: 1.9832
Epoch: 204, Loss: 0.2486, Val_Loss: 1.9639
Epoch: 205, Loss: 0.2369, Val_Loss: 1.9785
Epoch: 206, Loss: 0.2447, Val_Loss: 1.9853
Epoch: 207, Loss: 0.2608, Val_Loss: 1.9925
Epoch: 208, Loss: 0.2561, Val_Loss: 2.0375
Epoch: 209, Loss: 0.2670, Val_Loss: 2.0646
Epoch: 210, Loss: 0.2179, Val_Loss: 2.0626
Epoch: 211, Loss: 0.2417, Val_Loss: 2.0616
Epoch: 212, Loss: 0.2838, Val_Loss: 2.0656
Epoch: 213, Loss: 0.2431, Val_Loss: 2.1099
Epoch: 214, Loss: 0.2358, Val_Loss: 2.1728
Epoch: 215,

KeyboardInterrupt: 

In [21]:
out=model(dataset[0]).cpu().detach().numpy()
threshold=0.95
out = np.where(out>threshold, 1,0).reshape(32,64)
print(out)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
#print(r_input[0,:,:,2])
print(r_output[0])

In [ ]:
print(r_output_pos[0])

In [ ]:
print(model)

In [ ]:
#################################################################################################################

In [ ]:
def gen_example(y,x,num=1):
    start_time = time.time()
    input=np.zeros([1,y,x,3])
    output=np.zeros([1,y,x])
    

    while num > 0 :
        add_num=0
        error=0
        seed=active_seed((y,x))
        active_origin=active_trans(seed,2,4)
        active,active_inout,active_routing,mid,distance,min_rpt,x1,x2,y1,y2=in_out_mid_gen(active_origin)
        start,end,mid1,mid2,line1,line2,line3=define_interconnect(mid,x1,x2,y1,y2)
            
        for i in range(5):
            divide, div_pos,line_num,online_min= divide_line(active,min_rpt,start,end,mid1,mid2,add_num)
            add_num+=1
            opt_pos,pos,rpt_dist,error=find_opt_delay(active,active_inout,divide,div_pos,online_min)
            if error==1:
                break
            dist=cal_rpt_dist(active,opt_pos,pos,start,end,rpt_dist)
            if np.max(dist)<=100:
                num -= 1
                out=np.zeros(active.shape).astype(np.int8)
                for i in range(opt_pos.shape[0]):

                    xx=opt_pos[i][0]
                    yy=opt_pos[i][1]
                    out[yy][xx]=1
                out=out.reshape([1,y,x])

                temp=np.zeros([1,y,x,3])

                temp[0,:,:,0] = active.reshape(1,active.shape[0],active.shape[1])
                temp[0,:,:,1] = active_inout.reshape(1,active.shape[0],active.shape[1])
                temp[0,:,:,2] = active_routing.reshape(1,active.shape[0],active.shape[1])
                input=np.append(input,temp,axis=0)
                output = np.append(output,out,axis=0)
#                print("Add_num:::::",add_num)
                break
            if i == 5:
                if np.max(dist)>100:
                    error =1 
                    print("######################ERROR###############")
                

    output = np.delete(output,0,axis=0).astype(np.int8)
    input = np.delete(input,0,axis=0).astype(np.int8)
    np.save("./gnn/dataset/input.npy",input)
    np.save("./gnn/dataset/output.npy",output)
    end_time = time.time()
    print("Total time:",int(end_time-start_time),"s")
    return input,output,dist
        


In [ ]:
input,output,dist=gen_example(32,64,1000)


In [ ]:
print(input.dtype)
a=np.array([0],dtype=int)
print(a.dtype)

In [ ]:
b=np.copy(input[0][:,:,2])
a=np.where(output[0]==1)
print(a[0])
print(a[1])
for i in range(a[0].shape[0]):
    yy=a[0][i]
    xx=a[1][i]
    b[yy][xx]=3
print(b)
    

In [ ]:
#np.save("./dataset2/1000/input100k.npy",r_input)
#np.save("./dataset2/1000/output100k.npy",r_output)
r_input=np.load("./dataset2/1000/input100k.npy")
r_output=np.load("./dataset2/1000/output100k.npy")
print(r_input.shape)
print(r_output.shape)

In [ ]:
tt=np.zeros([100000,16384]).astype(np.int8)
for i in range(tt.shape[0]):
    tt[i]=r_output[i].flatten()

print(tt.shape)


In [ ]:
#resnet50 = tf.keras.applications.resnet.ResNet50(weights='imagenet', input_shape = (128,128,3),include_top=False)
#resnet50.summary()

eff_net=tf.keras.applications.EfficientNetV2S(weights="imagenet",input_shape=(128,128,3),include_top=False)
eff_net.summary()


In [ ]:

model3 = tf.keras.Sequential([
    eff_net,
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='sigmoid')
])

#model3.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
model3.summary()



In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
save = tf.keras.callbacks.ModelCheckpoint('best_model_2D.h5', monitor='val_loss', mode='min', save_best_only=True)
callback = [es, save]

model3.fit(r_input,tt2, epochs=200, batch_size = 128, validation_split =0.3, callbacks=callback )    
model3.save("./weights_resnet_2D.h5")

In [ ]:
del model3
for i in range(20):
    gc.collect()
    K.clear_session()
    tf.keras.backend.clear_session()
    torch.cuda.empty_cache()

In [ ]:
#model3 = tf.keras.models.load_model ('./weights_resnet.h5')
#model3 = tf.keras.models.load_model ('./best_model.h5')

pred=model3.predict(r_input)
total = np.zeros(pred.shape[0])
ans=np.where(pred>0.4,1,0)
true=0
false=0
for i in range(ans.shape[0]):
    verify1=np.isclose(tt2[i],ans[i],atol=0.1)
    verify2=np.where(verify1==False)[0]
    if verify2.shape==(0,):
        total[i]=True
        true += 1
    else :
        total[i]=False
        false += 1
print("True:",true)
print("Flase:",false)
print("Accuracy:",(true/(true+false))*100,"%")

In [ ]:
print(pred[5].reshape(64,64))

In [ ]:
def verify(model,threshold=0.4):
    true=0
    false=0
    input,output,cal=gen_example(64,64,bound=5,num=100)
    out=np.zeros([100,4096]).astype(np.int8)
    for i in range(output.shape[0]):
        out[i]=output[i].flatten()
    pred=model.predict(input)
    pred=np.where(pred > threshold,1,0)
    false_all = np.zeros(output.shape[0])
    
    for i in range(pred.shape[0]):
        verify1=np.isclose(out[i],pred[i],atol=0.1)
        verify2=np.where(verify1 == False)[0]
        if verify2.shape == (0,):
            total[i]=True
            true += 1
        else :
            total[i]=False
            false += 1
            false_all[i]=1
    #torch.cuda.empty_cache()
    
    return true, false, false_all, input, output, pred    
    

In [ ]:
true, false, false_all, test, test_out, pred = verify(model3, 0.35)

In [ ]:
print("True:",true)
print("False:",false)
print("Accuracy:", (true/(true+false))*100,"%")

In [ ]:
print(test[60][:,:,2])

In [ ]:
print(active_origin[0])